In [1]:
import obspy
import obspy.signal.filter
from obspy.io.sac.sactrace import SACTrace
from obspy.io.sac.util import get_sac_reftime
from obspy.clients.iris import Client
from obspy.taup import TauPyModel
import os
import glob
import numpy as np
import shutil
import pickle
from obspy import read
import random
import pandas as pd
import sys
import re

In [2]:
os.chdir("/mnt")
ddir = f"/mnt/c/Users/youzh/Documents/My documents(not synced)/sP_Workflow"
data_sac = f"{ddir}/Download/SAC"
wdir = f"{ddir}/Download/Verified"
jdir = f"{ddir}/judge"
wsl_dir = f"/home/youzhi/python-projects/sp-project"
os.getcwd()

'/mnt'

In [3]:
client = Client()
hk_model = f"{ddir}/hk.npz"
model = TauPyModel(model= hk_model)

# Convert to envelope, normalize and cut waveform

In [4]:
df_verified = pd.read_csv(f"{wdir}/verified.txt", names=["evt_id","sta", "Tp", 
                                                   "p_len","TsPmP", "sPmP_len",
                                                   "TsPn", "sPn_len", "quality", "evt_num"], header = None)

df_verified['evt_num'] = df_verified['evt_num'].astype(str)
df_verified

,evt_id,sta,Tp,p_len,TsPmP,sPmP_len,TsPn,sPn_len,quality,evt_num
0,39909423,CI.HOL.,15.558,0.3,18.173,0.3,nan,0.4,a,20220112002458770
1,39909511,AZ.LVA2.,3.723,0.2,11.662,0.3,nan,0.4,b,20220112021208390
2,39909511,AZ.SMER.,12.748,0.2,16.068,0.3,nan,0.4,b,20220112021208390
3,39909511,BC.RMX.,16.035,0.3,19.083,0.3,18.483,0.3,a,20220112021208390
4,39909663,CI.EML.,18.840,0.4,23.763,0.4,22.793,0.4,a,20220112090603170
...,...,...,...,...,...,...,...,...,...,...
122,40207208,CI.DZA.,20.400,0.4,22.900,0.4,22,0.3,c,20220314054039570
123,40207216,CI.THM.,11.900,0.4,15.800,0.4,nan,0.3,a,20220314055945990
124,39969103,BC.RMX.,8.550,0.4,13.700,0.4,nan,0.3,a,20220316054313670
125,39969135,CI.SLB.,15.400,0.4,19.100,0.4,nan,0.3,b,20220316060858910


In [5]:
evt_dir  = f"{ddir}/Download/events.txt"

events = pd.read_csv(evt_dir, header = None, delim_whitespace=True)


# Function to remove non-numerical symbols from a string
def remove_non_numerical(s):
    return re.sub('[^0-9]', '', s)

# Apply the function to the 'col1' column of the DataFrame
events[3] = events[3].apply(remove_non_numerical)
events

,0,1,2,3,4,5,6,7,8,9
0,39900807,eq,l,20220102001614080,32.2623,-115.3403,4.03,2.28,l,1.0
1,39901391,eq,l,20220102232745920,33.6480,-116.7418,15.41,2.36,l,1.0
2,39901519,eq,l,20220103023448200,35.2840,-119.3907,17.73,3.92,l,1.0
3,39902431,eq,l,20220104090504520,32.9600,-115.8628,7.95,2.00,l,1.0
4,39902479,eq,l,20220104124219810,34.1240,-117.5977,11.83,2.08,l,1.0
...,...,...,...,...,...,...,...,...,...,...
509,40292048,eq,l,20220627041323370,33.8412,-118.5292,10.77,2.27,l,1.0
510,40292088,eq,l,20220627065127410,32.6447,-115.7207,9.34,2.07,l,1.0
511,40292496,eq,l,20220627195131740,35.6527,-117.4580,1.33,2.10,l,1.0
512,40293376,eq,l,20220629041936690,33.1813,-115.6073,2.96,3.02,l,1.0


In [7]:
data_sac_b = "/home/youzhi/python-projects/sp-project/Data/BAD_SAC_for_readin"
data_sac_g = "/home/youzhi/python-projects/sp-project/Data/GOOD_SAC_for_readin"
switch = False

In [ ]:
# don't run this
# testing if the TsPmP values need to add origin time (sachd["o"])
if switch == True:

    data_sac = data_sac_g #good signals
else: 
    data_sac = data_sac_b #bad signals


for year in sorted(os.listdir(data_sac)):
    #print(year)
    inpath = os.path.join(data_sac, year)
    if os.path.isdir(inpath):
        #print(inpath)
        os.chdir(inpath)
        for bp in sorted(glob.glob("*.ev")):
            #print(bp)

            # calculate the envelope
            tr = obspy.read(bp)
            print(tr[0].stats.sac["nevid"])

In [8]:
# suppress warnings for nevid
import warnings
warnings.filterwarnings('ignore')

In [8]:
# this code plot will need to be run twice, once for bad and once for good
# remember to change the switch to False for the second run
if switch == True:

    data_sac = data_sac_g #good signals
else: 
    data_sac = data_sac_b #bad signals


for year in sorted(os.listdir(data_sac)):
    print(year)
    inpath = os.path.join(data_sac, year)
    if os.path.isdir(inpath):
        #print(inpath)
        os.chdir(inpath)
        for bp in sorted(glob.glob("*.bp.cut")):
            #print(bp)

            # calculate the envelope
            tr = obspy.read(bp)

            trenv = obspy.signal.filter.envelope(tr[0].data)
            
            if data_sac == data_sac_g:
                #good signals

                # add event id to ev
                evt_index = df_verified.loc[df_verified["evt_num"] == year].index[0]
                tr[0].stats.sac['nevid'] = str(df_verified["evt_id"][evt_index])

                #write sP arrival
                TsPmP = df_verified["TsPmP"][evt_index]
                Tp = df_verified["Tp"][evt_index]
                tr[0].stats.sac["t2"] = TsPmP - Tp #+ sachd["o"]
                
                tr[0].stats.sac["kt2"] = "1"
            else: 
                # bad signals
                tr[0].stats.sac["kt2"] = "0"
                evt_index = events.loc[events[3] == year].index[0]
                tr[0].stats.sac['nevid'] = str(events[0][evt_index])
                sachd = tr[0].stats.sac
                distaz = client.distaz(sachd["stla"], sachd["stlo"], sachd["evla"], sachd["evlo"])
                gcarc = distaz['distance']
                arrivals_spmp = model.get_travel_times(source_depth_in_km=sachd["evdp"],
                              distance_in_degree=gcarc, phase_list=['sPvmP'])
                arrivals_p= model.get_travel_times(source_depth_in_km=sachd["evdp"],
                              distance_in_degree=gcarc, phase_list=['ttp'])
                Tp = arrivals_p[0].time
                TsPmP = arrivals_spmp[0].time
                tr[0].stats.sac["t2"] = TsPmP - Tp


            # normalize the envelope
            trenv = trenv / np.max(trenv)
            tr[0].data = trenv
            

            # save ev file
            fname = year+'.' + tr[0].stats.network + '.' + tr[0].stats.station + tr[0].stats.location + '.BHZ'
            tr[0].write(fname +".ev", format='SAC')
            nbp = "{}.{}".format(fname,"ev")
            ntr = obspy.read(nbp)

            # cut the waveform within a given time window, -2 to 11 s
            # changed to 11 based on data I collected
            T = get_sac_reftime(ntr[0].stats.sac) + 0
            ntr.trim(starttime = T-2, endtime = T+11, fill_value=0)
            ntr[0].write(nbp, format='SAC')

            # tr = obspy.read(bp)
            # # cut the waveform within a given time window, -2 to 11 s
            # T = get_sac_reftime(tr[0].stats.sac) + 0
            # tr.trim(starttime = T-2, endtime = T+11, fill_value=0)
            # tr[0].write(bp, format='SAC')

20220112002458770
20220112021208390
20220112090603170
20220113032108850
20220113032453520
20220113082309790
20220115161902000
20220116091814410
20220116124930000
20220116145950590
20220117000229990
20220117155620910
20220118073915120
20220118084437210
20220120095230400
20220122010142300
20220122234305360
20220123044939870
20220123205715240
20220124023838010
20220125172319330
20220125192818690
20220125212130440
20220126050555330
20220126095905010
20220126103830020
20220127033401460
20220127095426310
20220128104007860
20220128203303970
20220129055641000
20220131234528720
20220201012424220
20220201192630930
20220202070130500
20220202100105740
20220203075810850
20220203151832540
20220203170822680
20220204105219770
20220205074104630
20220205074909660
20220205183420810
20220205234904960
20220206232447510
20220207121028870
20220209014742470
20220209040551690
20220209121834630
20220211121842170
20220211141550450
20220211142113660
20220211232548160
20220212232629910
20220214164135260
2022021417

# Move ev files into a new folder

In [9]:
target_dir_b = "/home/youzhi/python-projects/sp-project/Data/bad_readindata"
target_dir_g = "/home/youzhi/python-projects/sp-project/Data/good_readindata"
# good_readindata for good data
# bad_readindata for bad data

# move all the ev files into a folder
data_sac_b = "/home/youzhi/python-projects/sp-project/Data/BAD_SAC_for_readin"
data_sac_g = "/home/youzhi/python-projects/sp-project/Data/GOOD_SAC_for_readin"

In [10]:
switch = True
if switch == True:

    data_sac = data_sac_g
    target_dir = target_dir_g
else: 
    data_sac = data_sac_b
    target_dir = target_dir_b

num_files = 0
for path in os.listdir(data_sac):
    #print(path)
    if path.endswith("ini") != True:
        
        for file in os.listdir(os.path.join(data_sac, path)):
            #print(file)
            if file.endswith(".ev"):

                #delete rundundant files from previous runs
                #os.remove(os.path.join(data_sac, path, file))
                filepath = os.path.join(data_sac, path, file)
                shutil.copy(filepath, os.path.join(target_dir, file))
                #print(filepath)
	    #some times if might count desktop.ini files
                num_files += 1
print(num_files)

127


# Combine envelope for neural network readin

In [11]:
switch = True

datadir_b = "/home/youzhi/python-projects/sp-project/Data/bad_readindata"
datadir_g = "/home/youzhi/python-projects/sp-project/Data/good_readindata"

if switch == True:

    datadir = datadir_g
else: 
    datadir = datadir_b

signal_evid = []
signal_evlo = []
signal_evla = []
signal_stlo = []
signal_stla = []
signal_fname = []
envelop_signal = []
PmP_label = []
PmP_time = []
PmP_ptime = []
PmP_dist = []
PmP_evdp = []
PmP_mag = []
j=0
for file in os.listdir(datadir):
    if file.endswith(".ev"):
        j=j+1
        st = read(os.path.join(f"{datadir}", file))
        envelop_signal.append(st[0].data)
        filenm = file.split('.',6)
        filenmall=f"{filenm[0]}"+'.'+f"{filenm[1]}"+'.'+f"{filenm[2]}"
        signal_evid.append(st[0].stats.sac.nevid)
        signal_evlo.append(st[0].stats.sac.evlo)
        signal_evla.append(st[0].stats.sac.evla)
        signal_stlo.append(st[0].stats.sac.stlo)
        signal_stla.append(st[0].stats.sac.stla)
        signal_fname.append(filenmall)
        quality = int(st[0].stats.sac.get("kt2"))
        PmP_label.append(quality)
        PmP_time.append(st[0].stats.sac.t2)       ### differential traveltime between P and PmP waves
        PmP_ptime.append(st[0].stats.sac.o)       ### (negative) traveltime of first P wave (This is because waveform has been aligned to P-wave arrival)
        PmP_dist.append(st[0].stats.sac.dist)
        PmP_evdp.append(st[0].stats.sac.evdp)
        PmP_mag.append(st[0].stats.sac.mag)
print(j)
        #print("%d  %s  %g  %g  %g  %g  %g %d"%(j,signal_fname[j-1],PmP_time[j-1],PmP_ptime[j-1],PmP_dist[j-1],PmP_evdp[j-1],PmP_mag[j-1],PmP_label[j-1]))

127


# save into a single file

In [10]:
def saveVariables(variables, filename): #where 'variables' is a list of variables
    with open(filename, 'wb+') as file:
        pickle.dump(variables, file)

In [13]:
#fast save and retrieve data, no need to read in data again(very slow)

if switch == True:
    saveVariables([envelop_signal, PmP_time, PmP_label, PmP_dist, PmP_evdp, 
               PmP_mag, signal_stlo, signal_stla, signal_evlo, signal_evla, signal_evid, signal_fname], f"{wsl_dir}/Data/combined_readindata/g_readindata")
else:
    saveVariables([envelop_signal, PmP_time, PmP_label, PmP_dist, PmP_evdp, 
               PmP_mag, signal_stlo, signal_stla, signal_evlo, signal_evla, signal_evid, signal_fname], f"{wsl_dir}/Data/combined_readindata/b_readindata")


# Combine bad and good data

In [13]:
data_dir =  "/home/youzhi/python-projects/sp-project/Data/combined_readindata"
spdir = "/home/youzhi/python-projects/sp-project/Data"

def retrieveVariables(filename):
    variables = []
    with open(str(filename), 'rb') as file:
        variables = pickle.load(file)
    return variables


# read in data

[envelop_signal, PmP_time, PmP_label, 
 PmP_dist, PmP_evdp, PmP_mag, PmP_stlo, 
 PmP_stla, PmP_evlo, PmP_evla, PmP_evid, 
 PmP_fname] = retrieveVariables(f"{data_dir}/g_readindata")

[envelop_signal2, PmP_time2, PmP_label2, 
 PmP_dist2, PmP_evdp2, PmP_mag2, PmP_stlo2, 
 PmP_stla2, PmP_evlo2, PmP_evla2, PmP_evid2, 
 PmP_fname2] = retrieveVariables(f"{data_dir}/b_readindata")

# randomly pick 300 indexes between 0 and 4000

list1 = [envelop_signal, PmP_time, PmP_label, 
 PmP_dist, PmP_evdp, PmP_mag, PmP_stlo, 
 PmP_stla, PmP_evlo, PmP_evla, PmP_evid, 
 PmP_fname] 

indexes = np.random.choice(2348, 600, replace=False)

list2 = [envelop_signal2, PmP_time2, PmP_label2, 
 PmP_dist2, PmP_evdp2, PmP_mag2, PmP_stlo2, 
 PmP_stla2, PmP_evlo2, PmP_evla2, PmP_evid2, 
 PmP_fname2]

list3 = []

for i in range(len(list2)):
    list2[i] = np.array(list2[i])[indexes]
    
    list3.append(np.concatenate((list2[i], list1[i])))

saveVariables(list3, f"{spdir}/combined_readindata/c_readindata")


In [12]:
len(list3[0])

627